In [ ]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Imports

In [231]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit
from scipy.signal import savgol_filter, butter

In [7]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [8]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [102]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/0.2/data_fct_Adrien_point_by_point

In [12]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [13]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : D:/CIMLuser/adrien/Simulations/20200610/DC09_RF11/Try00
> Filename : Temp_SimuType0_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [2, 11, 18, 30, 39, 49]
> slashcond | 4
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 1


In [14]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC09_RF11']
[4]
['DC', []]
> condition names ['DC', 'RF']
> number of points 1
000 > DC09_RF11
> N_ions = 1024
> e_GMol = 50


In [15]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord,condition_parameters,slash_cfg,mode='GMol') # stopping_power

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 1
> Simulations pour chaque point | ['Try00']
Hello
['Temp_SimuType0_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG.dat'
 'Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType6_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG.dat'
 'Temp_SimuType6_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG50eV.dat']
0
Point n° 0
00 - 00  >  {}\D:/CIMLuser/adrien/Simulations/20200610\DC09_RF11\Try00
Time elapsed:  0.8573808000000014 s
Time elapsed:  0.014289680000000023 m


In [16]:
# Where SNR for eeach try in each  are compared to a threshold
# If SNR > some_threshold , we count it as a detection
# Then for each point the ratio number_of_detection/number_of_try is calculated
# This detection efficiency 

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(mean(trig_detection,axis=1))

print(len(trig_detection))

[1.]
1


In [17]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


# Plots
Nuage d'ions (XYZ)

## Visualise one specific point, and one case

In [18]:
# Choose point to plot

plot_point = 0
print(list(points_and_coord.keys())[plot_point]) # point condition
plot_try = 0
plot_subdir = all_subdir[plot_point+plot_try%13] # point address
print(plot_subdir)

DC09_RF11
D:/CIMLuser/adrien/Simulations/20200610\DC09_RF11\Try00


In [ ]:
# plot ion cloud XYZ

print(plot_subdir)
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
file_name = '{}/xva{}'.format(plot_subdir,sort(onlyfiles)[0].strip('.dat')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(plot_point)+'_try'+str(plot_try),fig_title='XYZ')

In [19]:
# Plot T and fluo PM

onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
ta, te, ti, to, tu = plot_T_and_PM_Init_Inje_Evol( plot_subdir+'/',sort(onlyfiles)[0].strip('.dat')[4:],1,'coucou' )

# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

# Plot gradient temperature

In [22]:
file_dir2 = 'D:/CIMLuser/adrien/Simulations/20200610/DC09_RF11/Try00/'

file_name = sort(onlyfiles)[0].strip('.dat')
i_aux = file_name.find('_N')
file1 = 'SimuType0'    + file_name[i_aux:] ########## file1 = 'SimuType0'    + file_name[i_aux:17+36]
file2 = 'SimuType4_01' + file_name[i_aux:]
file3 = 'SimuType2_01' + file_name[i_aux:]
file4 = 'SimuType6_01' + file_name[i_aux:]

tt1, T_CM1, T_aux1, PM1 = load_T_and_PM_simu(file_dir2+'Temp_'+file1)
tt2, T_CM2, T_aux2, PM2 = load_T_and_PM_simu(file_dir2+'Temp_'+file4+'50eV')
tt3, T_CM3, T_aux3, PM3 = load_T_and_PM_simu(file_dir2+'Temp_'+file3+'50eV')
tt4, T_CM4, T_aux4, PM4 = load_T_and_PM_simu(file_dir2+'Temp_'+file4+'50eV')

aux = mean(PM1[-100:])
PM_variation = ( aux - mean(PM3[-100:]) ) / aux
T_variation  = mean(T_aux3[-100:,0]) + mean(T_aux3[-100:,1]) + mean(T_aux3[-100:,2])

In [29]:
# Auxiliary arrays:
t_aux1 = array([tt2[ 0],tt2[ 0]])
t_aux2 = array([tt2[-1],tt2[-1]])
y1_aux = array([1.0e-3 ,1.0 ])
#     y2_aux = array([0 ,20 ])
y2_aux = array([0 ,50 ])

tt    = concatenate( (   tt1,   tt2,   tt3) )
T_aux = concatenate( (T_aux1,T_aux2,T_aux3) )
PM    = concatenate( (PM1,PM2,PM3) )

tt_relax    = concatenate( (   tt1,   tt4) )
T_aux_relax = concatenate( (T_aux1,T_aux4) )
PM_relax    = concatenate( (PM1,PM4) )


#fig_name = file_name[-9:]
figure('without laser ?'); clf()
ax1 = subplot(311)
semilogy(tt*1.e3,T_aux[:,0], label='Tx')
semilogy(tt*1.e3,T_aux[:,1], label='Ty')
semilogy(tt*1.e3,T_aux[:,2], label='Tz')
semilogy(t_aux1*1.e3,y1_aux,'r')
semilogy(t_aux2*1.e3,y1_aux,'r')
ax1.grid()
# annotate('Laser ON', xy=(0.5,350), xycoords='data',
    # size=24, ha='left', va='top', color='xkcd:azul',
    # bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))


legend(title='T Laser ON')
# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = subplot(312,sharex=ax1)
semilogy(tt_relax*1.e3,T_aux_relax[:,0], label='Tx')
semilogy(tt_relax*1.e3,T_aux_relax[:,1], label='Ty')
semilogy(tt_relax*1.e3,T_aux_relax[:,2], label='Tz',marker='+')
semilogy(t_aux1*1.e3,y1_aux,'r')
semilogy(t_aux2*1.e3,y1_aux,'r')
ax2.grid()
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
    # size=24, ha='left', va='top', color='xkcd:azul',
    # bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))
legend(title='T Laser OFF après injection')    
plt.setp(ax2.get_xticklabels(),visible=False)

ax3 = subplot(313,sharex=ax1)
plot(tt*1.e3,PM[:])
plot(t_aux1*1.e3,y2_aux,'r')
plot(t_aux2*1.e3,y2_aux,'r')
ax3.grid()

xlabel('time[ms]')
ylabel('Counts')


plt.tight_layout()
subplots_adjust(hspace=0.015)

In [106]:
# Plot T and fluo PM for RF relax

onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
ta, te, ti, to = plot_T_and_PM_Init_RFrelax_AfterCooling_Evol( plot_subdir+'/',sort(onlyfiles)[0].strip('.dat'),1,'coucou' )
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

In [221]:
te_smooth = [[],[],[]]
for k in range(3):
    te_smooth[k] = savgol_filter(te[:,k],window_length=701,polyorder=1)

C:\Anaconda3\envs\py37\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [230]:
figure('temperature gradient'); clf()
ax1 = subplot(211)
ax1.plot(ta,te[:,0],color='blue')
ax1.plot(ta,te[:,1],color='red')
ax1.plot(ta,te[:,2],color='green',marker='+')
#semilogy(ta,te[:,0],color='blue')
#semilogy(ta,te[:,1],color='red')
#semilogy(ta,te[:,2],color='green')
ax1.plot(ta,te_smooth[2],color='xkcd:black')
#semilogy(ta,te_smooth[2],color='xkcd:black')
ax1.grid()
ax1.set_ylabel('T [K]')
# annotate('Laser ON', xy=(0.5,350), xycoords='data',
    # size=24, ha='left', va='top', color='xkcd:azul',
    # bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

te_smooth_grad = gradient(te_smooth[k])/(ta[5]-ta[4])

ax2 = subplot(212,sharex=ax1)
ax2.plot(ta,te_smooth_grad)
ax2.plot(ta,savgol_filter(te_smooth_grad,window_length=101,polyorder=1),color='k')
ax2.grid()
ax2.set_ylabel('$\Delta T / s$ [K/s]')
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
    # size=24, ha='left', va='top', color='xkcd:azul',
    # bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))
ax2.set_xlabel('time [s]')

ax1.set_xlim(0.046,0.051)
ax1.set_ylim(-50,1000)
ax2.set_ylim(-1e6,1e6)
plt.tight_layout()
subplots_adjust(hspace=0.015)

C:\Anaconda3\envs\py37\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [249]:
figure('temperature gradient in fct of T'); clf()
ax1 = subplot(111)

te_smooth_grad = gradient(te_smooth[k])/(ta[5]-ta[4])
ax1.plot(te_smooth[k],te_smooth_grad,lw=0.5,marker='+')
#ax1.plot(te[:,2],savgol_filter(te_smooth_grad,window_length=101,polyorder=1),color='k')
ax1.set_xlabel('T [K]')
ax1.set_ylabel('$\Delta T / s$ [K/s]')
# annotate('Laser ON', xy=(0.5,350), xycoords='data',
    # size=24, ha='left', va='top', color='xkcd:azul',
    # bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')

#ax1.set_xlim(0.046,0.051)
#ax1.set_ylim(-50,1000)
#ax2.set_ylim(-1e6,1e6)
ax1.grid()
plt.tight_layout()

# Température avec pas de temps réaliste

In [227]:
# Plot only T with simulation time step
fname = 'only T'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti)
ax.grid()

ax.set_xlabel('[ms]')
ax.set_ylabel('[cts/500ns]')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

In [ ]:
# Plot only T with custom time step (realistic PM behaviour)

time_step = 1e-3 # s
custom_pts = len(ta)*(ta[1]-ta[0])/time_step
print(custom_pts)
somme_pts = int( round( time_step/(ta[1]-ta[0]) ) )
print(somme_pts)

custom_t = np.linspace(1,ceil(max(ta)/time_step),custom_pts+1)
custom_t_offset = max(custom_t) - max(ta)/time_step
custom_t = custom_t - custom_t_offset
custom_fluo = np.zeros( int(ceil(custom_pts)) )

for j in range( int(ceil(custom_pts))-1 ):
    temp = []
    print(j)
    print(somme_pts*j,somme_pts*(j+1)-1)
    for k in range(somme_pts*j,somme_pts*(j+1)):
        temp.append(ti[-k-1])
    temp = sum(temp)
    custom_fluo[-1-j] = temp/max(ti)/15

fname = 'only T custom T step'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti,zorder=10,label='500 ns')
# for j in range( int(ceil(custom_pts)) ):
#     ax.plot(ta[::-1][somme_pts*j:somme_pts*(j+1)]*1000,ti[::-1][somme_pts*j:somme_pts*(j+1)])
ax.scatter(custom_t[1:],custom_fluo[1:]/max(custom_fluo)*max(ti),s=200,marker='+',lw=5,edgecolor='black',zorder=100,label='1 ms')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
ax.grid()


ax.set_xlim(1.1,5.98)
ax.set_xlabel('[ms]')
ax.set_ylabel('[cts]')
ax.legend(title='Time step')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

## Data visualisation for all points (SNR)

In [ ]:
# Voltages associated with each condition
# In names and files each voltage is associated with numbers
# For instance on epoint is labeled DC03_RF18
# You need to know which voltage is DC03 and RF18

# You have to look on the Fortran code, example :
#    !dec$ elseif defined(Vrf18)
#    double precision  , parameter :: V_rf  = 7.179d1
#    character(len=100), parameter :: str_extra3 = 'q=0.666'
# Here RF18 is 71.79 V, q = 0.666

Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]
len(Udc),len(Urf)

In [ ]:
# A 3D visualisation of SNR for each point, each run
# Compulsory because this is where SNR is put in shape (under the name Z)

ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]

fname = '3D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111, projection='3d')

# fig1 = plt.figure(num=fname,clear=True)
# ##### PLOT DFluo et Proba détection
# ax1 = fig1.add_subplot(111)
# ax2 = ax1.twinx()
# clf()

l = 0
plot_Udc = []
plot_Urf = []
Z = []
for i,j in enumerate(points_and_coord):
    plot_Udc.append(Udc[int(points_and_coord[j][0])])
    plot_Urf.append(Urf[int(points_and_coord[j][1])])
    Z.append(Gmol_data[2][i])
            # ax.set_aspect('equal')
Z = np.array(Z)
print(shape(plot_Udc),shape(plot_Urf),shape(Z))

for k in range(13):
    ax.scatter(xs=plot_Udc,ys=plot_Urf,zs=Z[:,k],color='xkcd:azul',marker='o',s=10)
#             ax.plot(xs=r_LC_clip[0],ys=r_LC_clip[2],zs=r_LC_clip[1],marker='o',ms=0.45,ls='',color='xkcd:blue with a hint of purple')

#             ax.view_init(elev=18, azim=55)
#             ax.set_xlim(-3e-1,3e-1)
#             ax.set_ylim(-3e-1,3e-1)
#             ax.set_zlim(-3e-1,3e-1)

#             ax.text2D(0.05, 0.85, f'$U_{{ RF }}$ =  {Urf} V\n$U_{{DC}}$ = {Udc[j]} V', transform=ax.transAxes)
ax.set_xlabel('\n$U_{DC}$ [V]')
ax.set_ylabel('\n$U_{RF}$ [V]')
ax.set_zlabel('\n SNR ')

#         ax.plot(np.ones(100)*Udc[dc], np.ones(100)*Urf[rf], zs=linspace(0,0.8,100),color='black',ls=':',lw=1)

# X = [Udc for hhh in range(elem_1+1)]
# Y = [Urf for smeuiofh in range(elem_2-1)]
# # X, Y = np.meshgrid(X, Y)
# Z = asarray(transpose(mean_fluo))
# Z = np.meshgrid(Z)

# # surf = ax.plot_wireframe(transpose(X), Y, Z)
# surf = ax.plot_surface(transpose(X), Y, Z, rstride=1, cstride=1,
#                 cmap='viridis', edgecolor='black',alpha = 0.35)
# #         ax.errorbar(Udc[dc],mean_fluo[],yerr=prob_detect_xerr,xerr=None,
# #              elinewidth=2.5,capsize=None,color='xkcd:yellow orange',
# #              fmt=':+',ms=12,mew=3.5,mfc='None',mec='xkcd:yellow orange')
# cbar = plt.colorbar(surf)
# cbar.ax.set_ylabel(r'$\Delta F / F$ moyen ('+str(elem_0)+' essais)', rotation=90)


fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
# subplots_adjust(hspace=0.015)

# savefig(fname,dpi=300)

## 2D plot

In [ ]:
for i,j in enumerate(points_and_coord):
    print(j)
    print(points_and_coord[j][0],points_and_coord[j][1])

In [ ]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(Udc[int(points_and_coord[h][0])])
    y.append(Urf[int(points_and_coord[h][1])])
    z.append(mean(trig_detection[g]))
    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][:-15],fontsize=12)

im = ax.scatter(x,y,
        marker='o',s=100,edgecolor='black',
        c = z, cmap='rainbow')
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(SNR_trig_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)